# Assigment 4

In [27]:
#! pip install lxml
#! pip install unidecode

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

## Scrape all the results from "Extranjero" and get information from all tabs

# Click in all "Continentes", "Países" and "Estados" and extract table

In [27]:
driver = webdriver.Chrome( ChromeDriverManager().install() )

# Maximize window
driver.maximize_window()

# go to the link
driver.get( 'https://www.onpe.gob.pe/elecciones/historico-elecciones/' )

# Go to Results
driver.find_element_by_xpath( "/html/body/div/div/div/section[2]/div/div[2]/div/div/div[2]/div/div[2]/a").click()

# See all tabs
driver.window_handles

# Go to the second tab
driver.switch_to.window(driver.window_handles[1])

# Click on Results
driver.find_element_by_xpath( "/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[2]/div/div/a" ).click()
time.sleep(2)

#All selenium objetct in scope
scope = Select(driver.find_element_by_id( "select_ambito" ))
dict_scope_options = { option.text : option for option in scope.options }
dict_scope_options['EXTRANJERO'].click()

# Maximize window
#driver.maximize_window()
#time.sleep(5)   

# Store all_tables
all_tables = {}

# All selenium objects in CONTINENTES select
cont = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )

# Get number of total options
num_cont_options = len( cont.options )

for cont_idx in range( num_cont_options ):
    
    # Get again all CONTINENTES since HTML is refreshing
    # all elements
    
    cont = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )
    continente = cont.options[ cont_idx ]
    
    # Get CONTINENTE name
    cont_name = continente.text
    
    if cont_name != "--TODOS--" :
        
        # click on CONTINENTE
        continente.click()
        
        # Get all elements of PAÍS
        pais= Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
        num_pais_options = len( pais.options )
        
        for pais_idx in range( num_pais_options ):
            
            # Get again all PAISES since HTML is refreshing
            # all elements
            pais = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
            paises = pais.options[ pais_idx ]
                
            # Get PAISES name
            pais_name = paises.text
            
            if pais_name != "--TODOS--" :
                
                # click on PAISES
                paises.click()
                
                # Get all elements from ESTADO
                est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                num_est_options = len( est.options )
                
                for est_idx in range( num_est_options ):
                    
                    # Get again all ESTADO since HTML is refreshing
                    # all elements
                    est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                    estados = est.options[ est_idx ]
                    
                    # Get ESTADO name
                    est_name = estados.text
                    
                    if est_name != "--TODOS--" :
                        
                        # click on ESTADO
                        estados.click()
                        
                        # Get UBIGEO
                        ubigeo = driver.current_url.split("/")[ -1 ]
                        
                                                                  
                        ## Get table of presidential votes
                        # Get html at this point
                        table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
                        table_html = table_path.get_attribute( 'innerHTML' )
                        # Read the table using pandas
                        table = pd.read_html( table_html )
                        
                        # Cleaning tables
                        table[ 0 ].iloc[ 0 , 2 ] = 'political_parties'
                        row_new_columns = table[ 0 ].iloc[ 0 , 2: ]
                        clean_columns = row_new_columns.apply( lambda x : unidecode.unidecode( x ) ) \
                                              .str.replace( "TOTAL", "total") \
                                              .str.replace( "%VALIDOS", "share_valid") \
                                              .str.replace( "%EMITIDOS", "share_emited") \
                                              .tolist()
                        
                        # Selecting specific columns
                        table_clean = table[0].iloc[ 1:, 2: ].copy()
                        
                        # rename columns
                        table_clean.columns = clean_columns
                        
                        # New values to columns 
                        table_clean[ 'continent' ] = cont_name
                        table_clean[ 'country' ]   = pais_name
                        table_clean[ 'state' ]     = est_name
                        table_clean[ 'ubigeo' ]    = ubigeo
                        table_clean['link']        = driver.current_url
                        
                        # store tables
                        all_tables [ubigeo] = table_clean



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Robert\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-27-6b7d90e24499>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome( ChromeDriverManager().install() )
<ipython-input-27-6b7d90e24499>:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath( "/html/body/div/div/div/section[2]/div/div[2]/div/div/div[2]/div/div[2]/a").click()
<ipython-input-27-6b7d90e24499>:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath( "/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[2]/div/div/a" ).click()
<ipython-input-27-6b7d90e24499>:23: DeprecationWarning: find_element_

<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:64: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pais = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
<ipython-input-27-6b7d90e24499>:76: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead

<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.f

<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.f

<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.f

...

<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.f

<ipython-input-27-6b7d90e24499>:76: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:64: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.f

<ipython-input-27-6b7d90e24499>:76: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:64: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:83: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  est = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
<ipython-input-27-6b7d90e24499>:100: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table_path = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]" )
<ipython-input-27-6b7d90e24499>:64: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pais = Select( driver.

In [28]:
final_data = pd.concat( all_tables.values()).reset_index( drop = True )

In [29]:
final_data

,political_parties,total,share_valid,share_emited,continent,country,state,ubigeo,link
0,PARTIDO NACIONALISTA PERUANO,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
1,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
2,PARTIDO MORADO,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
3,PERU PATRIA SEGURA,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
4,VICTORIA NACIONAL,0,0.000%,0.000%,AFRICA,ARGELIA,ARGEL,910101,https://resultadoshistorico.onpe.gob.pe/EG2021...
...,...,...,...,...,...,...,...,...,...
3061,ALIANZA PARA EL PROGRESO,1,0.699%,0.617%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
3062,TOTAL DE VOTOS VÁLIDOS,143,100.000%,88.272%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
3063,VOTOS EN BLANCO,15,NaN,9.259%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...
3064,VOTOS NULOS,4,NaN,2.469%,OCEANIA,NUEVA ZELANDA,WELLINGTON,950202,https://resultadoshistorico.onpe.gob.pe/EG2021...


In [30]:
final_data.to_excel( r'..\..\..\Diplomado_PUCP\Lecture_7\_data_results\group_1_ass_4.xlsx' , index = False )